In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd

In [2]:
# load data

df=pd.read_csv('Annthyroid_02_v01.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att13,att14,att15,att16,att17,att18,att19,att20,att21,outlier
0,0.73,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.00060,0.015,0.120,0.082,0.146,0
1,0.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00025,0.030,0.143,0.133,0.108,0
2,0.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00190,0.024,0.102,0.131,0.078,0
3,0.64,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00090,0.017,0.077,0.090,0.085,0
4,0.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00025,0.026,0.139,0.090,0.153,0


In [3]:
df.shape

(6802, 22)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Annthyroid.json")
gc = GCForest(config)

In [15]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,21]
X = df[:,0:21]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
         # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-25 00:15:29,278][cascade_classifier.fit_transform] X_groups_train.shape=[(4557, 21)],y_train.shape=(4557,),X_groups_test.shape=[(2245, 21)],y_test.shape=(2245,)
[ 2018-04-25 00:15:29,280][cascade_classifier.fit_transform] group_dims=[21]
[ 2018-04-25 00:15:29,282][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:15:29,284][cascade_classifier.fit_transform] group_ends=[21]
[ 2018-04-25 00:15:29,285][cascade_classifier.fit_transform] X_train.shape=(4557, 21),X_test.shape=(2245, 21)
[ 2018-04-25 00:15:29,288][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4557, 21), X_cur_test.shape=(2245, 21)
[ 2018-04-25 00:15:29,957][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-25 00:15:30,805][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.12%
[ 2018-04-25 00:15:31,590][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 00:15:37,479][cascade_classifier.transform] X_groups_test.shape=[(2245, 21)]
[ 2018-04-25 00:15:37,481][cascade_classifier.transform] group_dims=[21]
[ 2018-04-25 00:15:37,483][cascade_classifier.transform] X_test.shape=(2245, 21)
[ 2018-04-25 00:15:37,485][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2245, 21)


Test Accuracy of ExtraTrees = 98.752784 %


In [10]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-25 00:15:39,003][cascade_classifier.transform] X_groups_test.shape=[(2245, 21)]
[ 2018-04-25 00:15:39,005][cascade_classifier.transform] group_dims=[21]
[ 2018-04-25 00:15:39,008][cascade_classifier.transform] X_test.shape=(2245, 21)
[ 2018-04-25 00:15:39,010][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2245, 21)


In [11]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-25 00:15:40,633][cascade_classifier.fit_transform] X_groups_train.shape=[(4557, 23)],y_train.shape=(4557,),X_groups_test.shape=[(2245, 23)],y_test.shape=(2245,)
[ 2018-04-25 00:15:40,635][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-25 00:15:40,637][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:15:40,638][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-25 00:15:40,640][cascade_classifier.fit_transform] X_train.shape=(4557, 23),X_test.shape=(2245, 23)
[ 2018-04-25 00:15:40,642][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4557, 23), X_cur_test.shape=(2245, 23)
[ 2018-04-25 00:15:41,292][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.68%
[ 2018-04-25 00:15:42,079][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.78%
[ 2018-04-25 00:15:42,937][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [12]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 00:15:48,796][cascade_classifier.transform] X_groups_test.shape=[(2245, 23)]
[ 2018-04-25 00:15:48,799][cascade_classifier.transform] group_dims=[23]
[ 2018-04-25 00:15:48,801][cascade_classifier.transform] X_test.shape=(2245, 23)
[ 2018-04-25 00:15:48,803][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2245, 23)


Test Accuracy of ExtraTrees = 99.153675 %


In [13]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [14]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-04-25 00:15:50,351][cascade_classifier.fit_transform] X_groups_train.shape=[(4557, 23)],y_train.shape=(4557,),X_groups_test.shape=[(2245, 23)],y_test.shape=(2245,)
[ 2018-04-25 00:15:50,354][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-25 00:15:50,355][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:15:50,357][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-25 00:15:50,359][cascade_classifier.fit_transform] X_train.shape=(4557, 23),X_test.shape=(2245, 23)
[ 2018-04-25 00:15:50,361][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4557, 23), X_cur_test.shape=(2245, 23)
[ 2018-04-25 00:15:50,983][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.46%
[ 2018-04-25 00:15:51,796][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.56%
[ 2018-04-25 00:15:52,625][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

('Test Accuracy of ExtraTrees = 99.153675 %', 0, 'fois')


[ 2018-04-25 00:16:00,669][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.46%
[ 2018-04-25 00:16:01,438][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.78%
[ 2018-04-25 00:16:02,280][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.56%
[ 2018-04-25 00:16:03,092][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.34%
[ 2018-04-25 00:16:03,912][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.34%
[ 2018-04-25 00:16:04,694][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=99.34%
[ 2018-04-25 00:16:05,529][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.34%
[ 2018-04-25 00:16:06,306][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.34%


('Test Accuracy of ExtraTrees = 99.064588 %', 1, 'fois')


[ 2018-04-25 00:16:10,357][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.68%
[ 2018-04-25 00:16:11,124][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.34%
[ 2018-04-25 00:16:11,893][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.78%
[ 2018-04-25 00:16:12,672][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.34%
[ 2018-04-25 00:16:13,512][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.34%
[ 2018-04-25 00:16:14,344][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=99.56%
[ 2018-04-25 00:16:15,212][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.34%
[ 2018-04-25 00:16:15,985][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.34%


...

('Test Accuracy of ExtraTrees = 98.886414 %', 17, 'fois')


[ 2018-04-25 00:18:38,401][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.46%
[ 2018-04-25 00:18:39,021][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.34%
[ 2018-04-25 00:18:39,743][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.12%
[ 2018-04-25 00:18:40,472][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.34%
[ 2018-04-25 00:18:41,209][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.56%
[ 2018-04-25 00:18:41,944][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=99.34%
[ 2018-04-25 00:18:42,679][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.78%
[ 2018-04-25 00:18:43,419][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.90%


('Test Accuracy of ExtraTrees = 99.064588 %', 18, 'fois')


[ 2018-04-25 00:18:47,127][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-25 00:18:47,851][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.34%
[ 2018-04-25 00:18:48,564][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.34%
[ 2018-04-25 00:18:49,265][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.34%
[ 2018-04-25 00:18:50,117][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.56%
[ 2018-04-25 00:18:50,919][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=99.12%
[ 2018-04-25 00:18:51,642][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.34%
[ 2018-04-25 00:18:52,380][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.68%


('Test Accuracy of ExtraTrees = 99.020045 %', 19, 'fois')


# ('Test Accuracy of ExtraTrees = 99.287305 %', 2, 'fois')